# Splunk App for Data Science and Deep Learning - Anomaly Detection with the PyOD library
## Example for ECOD: Unsupervised Outlier Detection Using Empirical Cumulative Distribution Functions

[PyOD is the most comprehensive and scalable Python toolkit for detecting outlying objects in multivariate data.](https://pyod.readthedocs.io/en/latest/index.html)

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with you Splunk platform by utilizing the Splunk App for Data Science and Deep Learning (DLTK). Find more examples and information in the app and on the [DLTK splunkbase page](https://splunkbase.splunk.com/app/4607/#/details).

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DLTK app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
from pyod.models.ecod import ECOD
#from pyod.models.iforest import IForest
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DLTK Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DLTK app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter.

### Option 1 - pull data from Splunk

In [ ]:
import libs.SplunkSearch as SplunkSearch

In [ ]:
search = SplunkSearch.SplunkSearch(search='| inputlookup hostperf.csv \n| eval _time=strptime(_time, "%Y-%m-%dT%H:%M:%S.%3Q%z") \n| timechart span=10m max(rtmax) as responsetime')

In [ ]:
df = search.as_df()
df

### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

| inputlookup hostperf.csv <br>
| eval _time=strptime(_time, "%Y-%m-%dT%H:%M:%S.%3Q%z") <br>
| timechart span=10m max(rtmax) as responsetime<br>
| fit MLTKContainer mode=stage algo=anomaly_detection_ecod responsetime into app:anomaly_detection_ecod_responsetime


After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("anomaly_detection_ecod")

In [ ]:
df

In [ ]:
param

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = ECOD(contamination=0.01)
    # parallization options for ECOD:
    # ECOD(n_jobs=2)    
    # most of other PyOD models would work similar, e.g. replace with Isolation Forest:
    #model = IForest()

    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    X = df[param['feature_variables'][0]]
    X_train = np.reshape(X.to_numpy(), (len(X), 1))

    # contamination = 0.01
    model.fit(X_train)
    
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    X = df[param['feature_variables'][0]]
    X_apply = np.reshape(X.to_numpy(), (len(X), 1))
    
    y_hat = model.predict(X_apply)  # outlier labels (0 or 1)
    y_scores = model.decision_function(X_apply)  # outlier scores

    result = pd.DataFrame(y_hat, columns=['outlier'])
    return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    if model is not None:
        if isinstance(model,ECOD):
            from joblib import dump, load
            dump(model, MODEL_DIRECTORY + name + '.joblib')
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    from joblib import dump, load
    model = load(model, MODEL_DIRECTORY + name + '.joblib')
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code

In [ ]:
# future: further explore SUOD
# please install suod first for SUOD by `pip install suod`
from pyod.models.suod import SUOD

# initialized a group of outlier detectors for acceleration
detector_list = [ECOD(), LOF(n_neighbors=35), IForest(n_estimators=100)]

# decide the number of parallel process, and the combination method
# then clf can be used as any outlier detection model
clf = SUOD(base_estimators=detector_list, n_jobs=2, combination='average', verbose=False)